In [123]:
# TODO 
# 4. create xmlifier function that accepts list
# 5. create jsonifier function that accepts list

def xmlifier(courses):

    for i in range(len(courses)):
        
        # And let's start populating the XML file by adding the scraped data one by one
        courseXML = SubElement(root, 'Course')

        courseNameXML = SubElement(courseXML, 'CourseName')
        courseNameXML.text = courseName

        courseDescriptionXML = SubElement(courseXML, 'CourseDescription')
        courseDescriptionXML.text = courseDescription

        courseSchoolXML = SubElement(courseXML, 'CourseSchool')
        courseSchoolXML.text = courseSchool

        courseSchoolXML = SubElement(courseXML, 'CourseCode')
        courseSchoolXML.text = courseCode

        # The following print lines are not needed since we will be saving to a file, 
        # but good to see the progress and for diagnostics
        print (courseName, i) 
        print (courseDescription)
        print (courseSchool)
        print (courseCode)
        print ("-------------")    


def getCityCourseCount(course):
    # Find the number of City University of London courses shown on website
    # by scraping tag course-finder__results__summary
    from bs4 import BeautifulSoup
    import urllib.request as ur
    from xml.etree.ElementTree import Element, SubElement, Comment, tostring
    from xml.dom import minidom
    urlToScrape = "https://www.city.ac.uk/courses?level=" + course
    r = ur.urlopen(urlToScrape).read()
    soup = BeautifulSoup(r, "lxml")
    pageCount = soup.find_all('div', attrs={'class': 'course-finder__results__summary'})
    for courseListItem in pageCount:
        try:
            # we are expecting a span tag like this "<span>67 search results</span>"            
            myspan = courseListItem.find('span').text
            # we've now stripped the tags and have "67 search results"
            # we split the string into an array, using the default white space delimiter
            # and take the zeroeth element which is 67
            mycount = myspan.split()[0]
        except Exception as e:
            mycount = "0" # no results
    return mycount

def getCourses(level, count):
    import re # regular expressions
    i = 1
    step = 10
    results = []
    while i <= int(count):
        # we know that the general URL format will be
        # https://www.city.ac.uk/courses?level=(level)&p=(n)
        # where paging is requested in querystring (text to the right of question mark)
        # attribute p, where (n) represents the starting number
        # we know that paging increments in counts of 10 i.e. 1, 11, 21 and so forth
        # hence we count up to 61, the url then requests pages starting from 61, we know
        # that our count for undergraduate courses was 67 can we can stop there
        # url = "https://www.city.ac.uk/courses?level=Undergraduate&p=" + str(i)
        url = "https://www.city.ac.uk/courses?level=" + level + "&p=" + str(i)
        r = ur.urlopen(url).read()
        soup = BeautifulSoup(r, "lxml")

        # get all the "div"s that ar of the class we are interested in.
        # note that the following line will return us a collection of "div" sections
        # courseList = soup.find_all('div', attrs={'class': 'course-finder__results__item course-finder__results__item--undergraduate'})
        # Let's use a wildcard instead
        courseList = soup.find_all('div', attrs={'class': re.compile('course-finder__results__item course-finder__results__item--.*')})
        
        for courseListItem in courseList:

            # Here we cover everything with try/except constructs to ensure that we are not failing when an element is not there.    
            try:
                # first go inside the DIV to access the course name, note that it is under a <a> tag, so that's where we need to access here.
                courseNameElement = courseListItem.find('div', attrs={'class': "col-sm-24 col-md-18 col-lg-20"})
                courseName = courseNameElement.find('a').text

            except Exception as e:
                courseName = ""

            try:
                # here, we access the course description, you can find that it is under a DIV
                courseDescriptionElement = courseListItem.find('div', attrs={'class': "course-finder__results__item__description"})
                courseDescription = courseDescriptionElement.text
            except Exception as e:
                courseDescription = ""

            try:
                # now on to scraping the name of the school offering the course, this time under an <a> tag 
                courseSchoolElement = courseListItem.find('div', attrs={'class': "course-finder__results__item__md course-finder__results__item__md--school"})
                courseSchool = courseSchoolElement.find('a').text
            except Exception as e:
                courseSchool = ""

            try:
                # when we try to get the course code, we notice that there are two <span> sections under the section 
                # we are interested in, we want to get the second one, hence => .find_all('span')[1]

                courseCodeElement_1 = courseListItem.find('div', attrs={'class': "course-finder__results__item__md course-finder__results__item__md--code"})
                courseCodeElement_2 = courseCodeElement_1.find_all('span')[2]
                courseCode = courseCodeElement_2.text
            except Exception as e:
                courseCode = ""
            # let's populate a dictionary for now, and decide how to format it later
            if len(courseName) > 0:
                course = {
                "name" : courseName,
                "description" : courseDescription,
                "school" : courseSchool,
                "code" : courseCode,
                "level" : level,
                }
                results.append(course)
        # increment count
        i += step
    return results

In [151]:
# Items in this list correspond to city url query string "level" e.g.
# https://www.city.ac.uk/courses?level=Executive Education
citycourselist = ["Undergraduate", "Postgraduate", "Research Degrees", "Foundation", "CPD", "Short Courses", "Executive Education"]
runningtotal = 0
results = []
for course in citycourselist:
    mycount = getCityCourseCount(course)
    results.extend(getCourses(course, mycount))
    print("Number of", course, "courses =", mycount)
    runningtotal += int(mycount)
print("Running total =", runningtotal)


Number of Undergraduate courses = 67
Number of Postgraduate courses = 142
Number of Research Degrees courses = 24
Number of Foundation courses = 9
Number of CPD courses = 251
Number of Short Courses courses = 133
Number of Executive Education courses = 15
Running total = 641


In [155]:
for i in range(len(results)):
    print(results[i]['name'])

Accounting and Finance 
Actuarial Science 
Adult Nursing 
Aeronautical Engineering 
Aeronautical Engineering 
Banking and International Finance 
Biomedical Engineering 
Biomedical Engineering 
Business Management 
Business Management, Digital Innovation and Entrepreneurship 
Business with Finance 
Business with Marketing 
Cass London Summer School 
Child Nursing 
Civil Engineering 
Civil Engineering 
Computer Science 
Computer Science 
Computer Science with Cyber Security 
Computer Science with Games Technology 
Computer Science with Games Technology 
Criminology 
Criminology and Psychology 
Criminology and Sociology 
Data Science 
Economics 
Economics with Accounting 
Electrical and Electronic Engineering 
Electrical and Electronic Engineering 
Engineering 
Engineering 
English 
Finance 
Financial Economics 
Health and Social Care 
History 
International Business 
International Political Economy 
International Politics 
International Politics and Sociology 
Investment and Financial Ri